In [3]:
from skorkart import scorecard_data

# Tüm hisseler tek Excel dosyasında, her biri ayrı sheet olarak:
results = scorecard_data(
    stock_codes=["THYAO", "GARAN"],
    start_period="2019/12",
    end_period="2025/03",
    lang="tr",
    save_to_excel=False,
    merge_to_single_excel=True
)

# Kodda DataFrame'lere doğrudan hisse adıyla erişebilirsiniz:
thy_financials = results["Financials"]["THYAO"]
garan_financials = results["Financials"]["GARAN"]

# Tüm finansalları dolaşmak için:
for hisse_adi, df in results["Financials"].items():
    print(f"{hisse_adi} finansalları:", df.head())

İşlem tamamlandı: THYAO
İşlem tamamlandı: GARAN
Tüm işlemler başarıyla tamamlandı.
THYAO finansalları:      Tarih  Net Satışlar Çeyrek (Mln TL)  \
0  2025/03                      176712.0   
1  2024/12                      193502.0   
2  2024/09                      221815.0   
3  2024/06                      182875.0   
4  2024/03                      147238.0   

   Net Satışlar (Yıllıklandırılmış) (Mln TL)  FAVÖK (Çeyrek) (Mln TL)  \
0                                   774904.0                  10167.0   
1                                   745430.0                  27603.0   
2                                   697545.0                  59088.0   
3                                   644821.0                  34065.0   
4                                   569579.0                  15500.0   

   FAVÖK (Yıllıklandırılmış) (Mln TL)  \
0                            130923.0   
1                            136256.0   
2                            130725.0   
3                            

In [22]:
from isyatirimhisse import fetch_stock_data, fetch_index_data, fetch_financials

df = fetch_financials(
    symbols="THYAO",
    start_year=2025,
    end_year=2025,
    exchange="TRY",
    financial_group='1',
    save_to_excel=False
)
df

,FINANCIAL_ITEM_CODE,FINANCIAL_ITEM_NAME_TR,FINANCIAL_ITEM_NAME_EN,2025/3,2025/6,SYMBOL
0,1A,Dönen Varlıklar,CURRENT ASSETS,352676000000,397076000000,THYAO
1,1AA,Nakit ve Nakit Benzerleri,Cash and Cash Equivalents,88821000000,98791000000,THYAO
2,1AB,Finansal Yatırımlar,Short-Term Financial Investments,120076000000,150560000000,THYAO
3,1AC,Ticari Alacaklar,Short-Term Trade Receivables,40800000000,40762000000,THYAO
4,1AD,Finans Sektörü Faaliyetlerinden Alacaklar,Short-Term Receivables from Financial Operations,0,0,THYAO
...,...,...,...,...,...,...
142,4CBH,Diğer Nakit Girişi/Çıkışı,Other Cash Inflow/Outflow,0,0,THYAO
143,4CBI,Nakit ve Benzerlerindeki Değişim,Change in Cash & Cash Equivalents,-8493000000,2626000000,THYAO
144,4CBJ,Diğer Nakit ve Nakit Benzerlerindeki Artış,Other Cash & Cash Equivalents Change,0,0,THYAO
145,4CBK,Dönem Başı Nakit Değerler,Cash at the beginning of the Quarter,90164000000,90164000000,THYAO


In [1]:
import pandas as pd
import numpy as np
import sqlite3
from isyatirimhisse import fetch_financials

conn = sqlite3.connect("C:/Users/KULLANICI/Desktop/portfolio-backtest-project/data/database.db")

ticker_dict = {"THYAO":1,"BIMAS":2}

df = fetch_financials(
    symbols=list(ticker_dict.keys()),
    start_year=2025,
    end_year=2025,
    exchange="TRY",
    financial_group='1',
    save_to_excel=False
)

df.drop(['FINANCIAL_ITEM_NAME_TR',"FINANCIAL_ITEM_CODE"], axis=1, inplace=True)  

df_melted = df.melt(
    id_vars=['FINANCIAL_ITEM_NAME_EN', 'SYMBOL'],
    var_name='Period',
    value_name='Value'
)

df_melted['Value'] = pd.to_numeric(df_melted['Value'], errors='coerce')

df_pivoted = df_melted.pivot_table(
    index=['SYMBOL', 'Period'],
    columns='FINANCIAL_ITEM_NAME_EN',
    values='Value'
).reset_index()
df_pivoted.columns.name = None
df_pivoted.columns = df_pivoted.columns.str.lower().str.replace(" ","_")
df_pivoted = df_pivoted.rename(columns={'net_sales': 'revenue',"gross_profit_(loss)": "gross_profit","operating_profits":"operating_profit","net_profit_after_taxes":"net_income","long_term_liabilities":"long_term_debt","short_term_liabilities":"short_term_debt","shareholders_equity":"equity"})
df_pivoted['ebitda'] = df_pivoted['operating_profit'] + df_pivoted['depreciation_&_amortization']
df_pivoted['fixed_assets'] = df_pivoted['total_assets'] - df_pivoted['current_assets']
df_pivoted['gross_debt'] = df_pivoted['short-term_financial_loans'] + df_pivoted['long-term_financial_loans']
df_pivoted['net_debt'] = df_pivoted['gross_debt'] - df_pivoted["cash\xa0and\xa0cash\xa0equivalents"]
df_pivoted['company_id'] = df_pivoted['symbol'].map(ticker_dict)
df_pivoted["date_of_publish"] = np.nan

df_pivoted = df_pivoted[['company_id',"period","date_of_publish","revenue","gross_profit","operating_profit","ebitda","net_income","current_assets","fixed_assets","long_term_debt","short_term_debt","gross_debt","net_debt","equity"]]

df_pivoted.to_sql('financial', conn, if_exists='append', index=False)
conn.close()


In [2]:
df_pivoted

,company_id,period,date_of_publish,revenue,gross_profit,operating_profit,ebitda,net_income,current_assets,fixed_assets,long_term_debt,short_term_debt,gross_debt,net_debt,equity
0,2,2025/3,NaN,1.477227e+11,2.585878e+10,-1.297669e+09,4.883661e+09,2.710405e+09,1.052041e+11,1.815128e+11,4.924420e+10,1.044967e+11,4.209025e+10,3.413588e+10,1.329760e+11
1,2,2025/6,NaN,3.098349e+11,5.889611e+10,2.401768e+09,1.471529e+10,5.553111e+09,9.922073e+10,1.962433e+11,5.409232e+10,1.046679e+11,4.646131e+10,4.354015e+10,1.367039e+11
2,1,2025/3,NaN,1.767120e+11,1.173100e+10,-2.879000e+09,1.753400e+10,-1.854000e+09,3.526760e+11,1.182270e+12,4.244820e+11,3.930280e+11,5.376700e+11,4.488490e+11,7.174360e+11
3,1,2025/6,NaN,4.080360e+11,5.614600e+10,2.452900e+10,6.748200e+10,2.493500e+10,3.970760e+11,1.255513e+12,4.688030e+11,4.316660e+11,5.845920e+11,4.858010e+11,7.521200e+11
